# Clean [Message Types Excel File](message_types.xlsx)
---
[GITHUB](https://github.com/PacktPublishing/Machine-Learning-for-Algorithmic-Trading-Second-Edition/blob/master/02_market_and_fundamental_data/01_NASDAQ_TotalView-ITCH_Order_Book/01_parse_itch_order_flow_messages.ipynb)

In [1]:
import pandas as pd 

# ITCH Format Settings
---
### From [message_types.xlsx](message_types.xlsx)
### [`struct`](https://docs.python.org/3/library/struct.html) module for binary data 
- ITCH tick data comes in binary format 
- `struct` parses binary data using format strings 
    - identifies the message element by indicating length and type of various components of the byte string 
    - conversions between Python values and C structs represented as Python byte objects 

### Defining Format Strings 

In [2]:
event_codes = {'O' : 'Start of Message', 
            'S' : 'Start of System Hours', 
            'Q' : 'Start of Market Hours',
            'M' : 'End of Market Hours', 
            'E' : 'End of System Hours', 
            'C' : 'End of Messages'}

encoding = {'primary_market_maker': {'Y': 1, 'N': 0},
        'printable'           : {'Y': 1, 'N': 0},
        'buy_sell_indicator'  : {'B': 1, 'S': -1},
        'cross_type'          : {'O': 0, 'C': 1, 'H': 2},
        'imbalance_direction' : {'B': 0, 'S': 1, 'N': 0, 'O': -1}}

# Assembles Format Strings According to the Formats Dictionary 
formats = {
    ('integer', 2): 'H',  #int of length 2 -> format string 'H'
    ('integer', 4): 'I',
    ('integer', 6): '6s',  #int of length 6 -> parse as string, convert later
    ('integer', 8): 'Q',
    ('alpha',   1): 's',
    ('alpha',   2): '2s',
    ('alpha',   4): '4s',
    ('alpha',   8): '8s',
    ('price_4', 4): 'I',
    ('price_8', 8): 'Q',
}

## Create Message Specs for Binary Data Parser 
### Load Message Types 
- `message_types.xlxs` contains messasge type specs (per [ITCH Protocol Documentation](https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf))

In [3]:
message_data = (pd.read_excel('message_types.xlsx', sheet_name='messages')
                .sort_values('id')
                .drop('id', axis=1))


In [4]:
message_data.head()

,Name,Offset,Length,Value,Notes
0,Message Type,0,1,S,System Event Message
1,Stock Locate,1,2,Integer,Always 0
2,Tracking Number,3,2,Integer,Nasdaq internal tracking number
3,Timestamp,5,6,Integer,Nanoseconds since midnight
4,Event Code,11,1,Alpha,See System Event Codes below


### Clean Message Types
- function `clean_message_types()` runs basic cleaning steps 

In [5]:
def clean_message_types(df): 
    df.columns = [c.lower().strip() for c in df.columns]
    df.value = df.value.str.strip()
    df.name = (df.name
            .str.strip()
            .str.lower()
            .str.replace(' ','_')
            .str.replace('-','_')
            .str.replace('/','_'))
    df.notes = df.notes.str.strip()
    df['message_type'] = df.loc[df.name == 'message_type', 'value'] 
    return df 

In [6]:
message_types = clean_message_types(message_data)

### Get Message Labels
- extract message type codes and names to make results more readable 

In [7]:
message_labels = (message_types.loc[:, ['message_type', 'notes']]
                    .dropna()
                    .rename(columns={'notes':'name'}))

message_labels.name = (message_labels.name
                    .str.lower()
                    .str.replace('message','')
                    .str.replace('.','')
                    .str.strip().str.replace(' ','_'))

message_labels.head()

/var/folders/m_/kvbpx92x4xgbpzq61g5jdz_h0000gq/T/ipykernel_28345/416926696.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  message_labels.name = (message_labels.name


,message_type,name
0,S,system_event
5,R,stock_directory
23,H,stock_trading_action
31,Y,reg_sho_short_sale_price_test_restricted_indic...
37,L,market_participant_position


### Finalize Specification Details 
- [Struct](https://docs.python.org/3/library/struct.html) Module: use format information to parse binary source data 
- Messages consist of fields defined by offset, length, and type of value

In [8]:
message_types.message_type = message_types.message_type.ffill()
message_types = message_types[message_types.name != 'message_type']
message_types.value = (message_types.value
                        .str.lower()
                        .str.replace(' ', '_')
                        .str.replace('(','')
                        .str.replace(')',''))

message_types.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 1 to 172
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          152 non-null    object
 1   offset        152 non-null    int64 
 2   length        152 non-null    int64 
 3   value         152 non-null    object
 4   notes         152 non-null    object
 5   message_type  152 non-null    object
dtypes: int64(2), object(4)
memory usage: 8.3+ KB


/var/folders/m_/kvbpx92x4xgbpzq61g5jdz_h0000gq/T/ipykernel_28345/3920762019.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  message_types.value = (message_types.value
/var/folders/m_/kvbpx92x4xgbpzq61g5jdz_h0000gq/T/ipykernel_28345/3920762019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  message_types.value = (message_types.value


In [9]:
message_types.head()

,name,offset,length,value,notes,message_type
1,stock_locate,1,2,integer,Always 0,S
2,tracking_number,3,2,integer,Nasdaq internal tracking number,S
3,timestamp,5,6,integer,Nanoseconds since midnight,S
4,event_code,11,1,alpha,See System Event Codes below,S
6,stock_locate,1,2,integer,Locate Code uniquely assigned to the security ...,R


---
### Save Cleaned `message_types.xlsx` to `.csv` File*

In [10]:
message_types.to_csv('message_types.csv', index=False)

---
